In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
path = './multilabel_modified/multilabel_classification(6)-reduced_modified.csv'
df = pd.read_csv(path)
df

,Image_Name,"Classes(motorcycle, truck, boat, bus, cycle, , , , , , , sitar, ektara, flutes, tabla, harmonium)",motorcycle,truck,boat,bus,cycle,sitar,ektara,flutes,tabla,harmonium
0,image1.jpg,bus,0,0,0,1,0,0,0,0,0,0
1,image2.jpg,sitar,0,0,0,0,0,1,0,0,0,0
2,image3.jpg,flutes,0,0,0,0,0,0,0,1,0,0
3,image4.jpg,bus,0,0,0,1,0,0,0,0,0,0
4,image5.jpg,bus,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7938,image7839.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1
7939,image7840.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1
7940,image7841.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1
7941,image7842.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1


In [3]:
image_path = './multilabel_modified/images/'

In [4]:
os.path.join(image_path, "image1.jpg")

'./multilabel_modified/images/image1.jpg'

In [5]:
def load_and_preprocess_image(img_name):
    img = cv2.imread(os.path.join(image_path, img_name))
    img = cv2.resize(img, (128, 128))  # Resize to 128x128
    img = img / 255.0  # Normalize the image to [0, 1]
    return img

In [6]:
images = np.array([load_and_preprocess_image(img_name) for img_name in df['Image_Name']])
labels = df.iloc[:, 2:].values

In [7]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [8]:
def model_creation():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(10, activation='sigmoid'))  # 10 classes with sigmoid for multi-label classification
    
    return model

model = model_creation()

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 flatten (

In [10]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    verbose=1)

Epoch 1/20
199/199 [==============================] - 28s 137ms/step - loss: 0.2382 - accuracy: 0.4411 - val_loss: 0.1618 - val_accuracy: 0.6218
Epoch 2/20
199/199 [==============================] - 26s 131ms/step - loss: 0.1478 - accuracy: 0.6517 - val_loss: 0.1382 - val_accuracy: 0.7231
Epoch 3/20
199/199 [==============================] - 27s 134ms/step - loss: 0.1086 - accuracy: 0.7369 - val_loss: 0.1257 - val_accuracy: 0.7030
Epoch 4/20
199/199 [==============================] - 26s 131ms/step - loss: 0.0808 - accuracy: 0.7940 - val_loss: 0.1182 - val_accuracy: 0.7199
Epoch 5/20
199/199 [==============================] - 26s 131ms/step - loss: 0.0558 - accuracy: 0.8492 - val_loss: 0.1144 - val_accuracy: 0.7470
Epoch 6/20
199/199 [==============================] - 26s 133ms/step - loss: 0.0401 - accuracy: 0.8738 - val_loss: 0.1226 - val_accuracy: 0.7514
Epoch 7/20
199/199 [==============================] - 26s 131ms/step - loss: 0.0302 - accuracy: 0.8941 - val_loss: 0.1346 - val_ac

In [11]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

50/50 [==============================] - 2s 30ms/step - loss: 0.1810 - accuracy: 0.7388
Validation Loss: 0.1810450553894043
Validation Accuracy: 0.7388294339179993
